In [132]:
import pandas as pd
import warnings
import json
warnings.simplefilter(action='ignore', category=FutureWarning)
with open('../keys.json') as json_file:
    column_keys = json.load(json_file)
terror_data = pd.read_csv('../data/globalterrorismdb_0221dist_filtered.csv')
disaster_data = pd.read_csv('../data/emdat_public_2022_01_31.csv')
selecteddisaster__data = [
  terror_data,
  disaster_data
]

/tmp/ipykernel_670406/2707642581.py:8: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  disaster_data = pd.read_csv('../data/emdat_public_2022_01_31.csv')


In [133]:
countries_terror = [
  'Canada',
  'United States',
  'Mexico',
  'India',
  'China',
  'Sudan',
  'Bangladesh',
  'Brazil',
  'Niger',
]
countries_full = [
  'United States of America (the)',
  'Canada',
  'Mexico',
  'India',
  'China',
  'Sudan',
  'Bangladesh',
  'Brazil',
  'Niger'
]

terror_columns = [
  'eventid',
  'iyear',
  'imonth',
  'iday',
  'End Year',
  'End Month',
  'End Day',
  'country_txt',
  'success',
  'attacktype1_txt',
  'nkill',
  'nwound',
  'propextent_txt',
]

disaster_columns = [
  'Dis No',
  'Country',
  'Start Year',
  'Start Month',
  'Start Day',
  'End Year',
  'End Month',
  'End Day',
  'Disaster Type',
  'Total Affected',
  'Total Deaths',
]

In [134]:
selected_teror_data = pd.DataFrame(columns = terror_columns)

for country in countries_terror:
    current_info = terror_data.loc[terror_data['country_txt'] == country]
    current_info['End Year'] = current_info['iyear']
    current_info['End Month'] = current_info['imonth']
    current_info['End Day'] = current_info['iday']
    current_info = current_info.loc[terror_data['iyear'].between(2005, 2020)]
    selected_teror_data = pd.concat((selected_teror_data, current_info[terror_columns]))


#ignore data with missing attack type
attack_type = (selected_teror_data[selected_teror_data['attacktype1_txt'] == "Unknown"].index) | selected_teror_data[selected_teror_data['attacktype1_txt'] == ""].index
selected_teror_data.drop(attack_type, inplace=True)

# only take data of sucessfull attacks (val = 1), so ignore val = 0
success = selected_teror_data[selected_teror_data['success'] == 0].index
selected_teror_data.drop(success, inplace=True)

# only take catastrophic and major property damage events
# damage = (selected_teror_data[selected_teror_data['propextent_txt'] == "Minor (likely < $1 million)"].index) | (selected_teror_data[selected_teror_data['propextent_txt'] == "Unknown"].index) | (selected_teror_data[selected_teror_data['propextent_txt'].isnull()].index) 
damage = (selected_teror_data[selected_teror_data['propextent_txt'] == "Unknown"].index) | (selected_teror_data[selected_teror_data['propextent_txt'].isnull()].index) 
selected_teror_data.drop(damage, inplace=True)
selected_teror_data = selected_teror_data.drop(["propextent_txt"], axis=1)

# only take events with at least 1 death or affected/wounded
deaths_and_affected = (selected_teror_data[selected_teror_data['nkill'].isnull()].index) | (selected_teror_data[selected_teror_data['nwound'].isnull()].index) | (selected_teror_data[selected_teror_data['nkill'] == 0].index) | (selected_teror_data[selected_teror_data['nwound'] == 0].index) 
selected_teror_data.drop(deaths_and_affected, inplace=True)
selected_teror_data = selected_teror_data.drop(["success"], axis=1)

selected_teror_data.rename(columns=column_keys, inplace=True)
terror_data = selected_teror_data.copy(True)
selected_teror_data.to_csv(path_or_buf=f'./Terrorism_seed.csv', index=False)


/tmp/ipykernel_670406/1834283556.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_info['End Year'] = current_info['iyear']
/tmp/ipykernel_670406/1834283556.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_info['End Month'] = current_info['imonth']
/tmp/ipykernel_670406/1834283556.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [135]:
selected_disaster_data = pd.DataFrame(columns = disaster_columns)

for country in countries_full:
    current_info = disaster_data.loc[disaster_data['Country'] == country]
    selected_disaster_data = selected_disaster_data.replace(['United States of America (the)'],'United States')
    current_info = current_info.loc[disaster_data['Start Year'].between(2005, 2020)]
    selected_disaster_data = pd.concat((selected_disaster_data, current_info[disaster_columns]))

# selected_disaster_data['Total Deaths'] = np.where((selected_disaster_data['Total Affected'] > 0) &  (selected_disaster_data['Total Deaths'].isna()), 0 , "" )
#Fills in missing values in Total affected and Total deaths based off assumptions.
selected_disaster_data.loc [(selected_disaster_data['Total Affected'] > 0) & (selected_disaster_data['Total Deaths'].isnull()), 'Total Deaths'] = 0
selected_disaster_data.loc [(selected_disaster_data['Total Affected'].isnull()) & (selected_disaster_data['Total Deaths'].isnull()), 'Total Deaths'] = 0 
selected_disaster_data.loc [(selected_disaster_data['Total Affected'].isnull()) & (selected_disaster_data['Total Deaths'].notnull()), 'Total Affected'] = selected_disaster_data['Total Deaths']

#Deletes columns with missing data in Start day
start_d = selected_disaster_data[selected_disaster_data['Start Day'].isnull()].index
selected_disaster_data.drop(start_d, inplace=True)

#Deletes columns with missing data in End day
end_d = selected_disaster_data[selected_disaster_data['End Day'].isnull()].index
selected_disaster_data.drop(end_d, inplace=True)

#Deletes columns with missing data in End Month
end_m = selected_disaster_data[selected_disaster_data['End Month'].isnull()].index
selected_disaster_data.drop(end_m, inplace=True)

selected_disaster_data.rename(columns=column_keys, inplace=True)

# selected_disaster_data = selected_disaster_data.rename(columns=column_keys)
disaster_data = selected_disaster_data.copy(deep=True)
selected_disaster_data.to_csv(path_or_buf=f'./Disaster_seed.csv', index=False)

In [136]:
t1 = disaster_data.reset_index(drop=True)
t2 = terror_data.reset_index(drop=True)
t1.columns = ['seed_id','start_year','end_month','end_day','end_year','end_month','end_day','country','event_type','total_affected','total_deaths']
t2.columns = ['seed_id','start_year','end_month','end_day','end_year','end_month','end_day','country','event_type','total_affected','total_deaths']
event_data = pd.concat([t1, t2],ignore_index=True)
event_data.insert(0, 'key', [i for i in range(len(event_data.index))])
event_data.to_csv(path_or_buf=f'./event_seed.csv', index=False)
